In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import csv

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
df

In [ ]:
train_df = df.loc[:20896]
val_df = df.loc[20897:].reset_index(drop = True)

In [ ]:
train_df['image_id'] = '../input/cassava-leaf-disease-classification/train_images'+os.sep + train_df['image_id']
val_df['image_id'] = '../input/cassava-leaf-disease-classification/train_images'+os.sep + val_df['image_id']

In [ ]:
train_files_ds = tf.data.Dataset.from_tensor_slices((train_df['image_id'],train_df['label']))
val_files_ds = tf.data.Dataset.from_tensor_slices((val_df['image_id'],val_df['label']))

In [ ]:
for i in train_files_ds:
    print(i)
    break

In [ ]:
def handle_filenames(filename,label):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image,channels = 3)
    image = tf.image.resize(image,size = (224,224))
    return image,label
auto = tf.data.experimental.AUTOTUNE

train_ds = train_files_ds.map(handle_filenames).batch(10).prefetch(auto)
val_ds = val_files_ds.map(handle_filenames).batch(10).prefetch(auto)

In [ ]:
def show(images,labels):
    plt.figure(figsize=(12, 12))
    for it, (image, label) in enumerate(zip(images[:10], labels[:10])):
        plt.subplot(4, 4, it+1)
        plt.imshow(image/255.0)
        plt.axis('off')
        plt.title({int(label)})
for i in train_ds.take(2):
    show(i[0],i[1])

In [ ]:
preprocess = keras.Sequential([
    layers.RandomFlip(),
    layers.RandomRotation(0.2),
    layers.RandomZoom((-0.2, 0)),
    layers.RandomContrast((0.2,0.2))
])

#vit_backbone = hub.KerasLayer("https://tfhub.dev/sayakpaul/vit_b8_fe/1", trainable=False) 
backbone = keras.applications.EfficientNetB3(
    include_top = False,
    weights = '../input/efficientnetb3notop/efficientnetb3_notop.h5',
    input_shape = (224,224,3)
)
resnet_preprocess = keras.applications.efficientnet.preprocess_input
backbone.trainable = True

In [ ]:
inp = keras.Input(shape = (224,224,3))
x = preprocess(inp)
x = resnet_preprocess(x)
x = backbone(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(5,activation = 'softmax')(x)
model = keras.Model(inp,x)
model.summary()

In [ ]:
LR_START = 0.0001
LR_MAX = 0.001 * strategy.num_replicas_in_sync
LR_MIN = 0.0001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [ ]:
decay_steps = int(round(len(train_df)/10))*10
cosine_decay = keras.experimental.CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3)

In [ ]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = keras.optimizers.Adam(cosine_decay),
    metrics = ['accuracy']
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5
)
model.fit(train_ds,
          validation_data = val_ds,
          epochs = 30,
          callbacks = [early_stopping])

In [ ]:
model.save('model.h5')

In [ ]:
l = os.listdir("../input/cassava-leaf-disease-classification/test_images")
parent = "../input/cassava-leaf-disease-classification/test_images/"
predictions = []
predictions.append(["image_id", "label"])
for i in l :
    child = parent + i
    img = cv2.imread(child)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    img = img.reshape((1, 224, 224, 3))
    pred = model.predict(img)
    pred = pred.reshape((5,))
    print(np.argmax(pred))
    del img
    predictions.append([i, str(np.argmax(pred))])
    del pred


with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(predictions)

In [ ]:
pd.read_csv('submission.csv')